# 检测器模块

## 数据集处理（转化为CocoVID格式）

### 训练集

In [1]:
!rm ./data/VisDrone_MOT/train/cocoformat.json

rm: cannot remove './data/VisDrone_MOT/train/cocoformat.json': No such file or directory


In [2]:
# convert the dataset to coco format
!python ./tools/convert_datasets/visdronemot/visdronemot2coco.py -d ./data/VisDrone_MOT/train

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
100%|███████████████████████████████████████████| 56/56 [00:08<00:00,  6.66it/s]


### 测试集

In [3]:
!rm ./data/VisDrone_MOT/test/cocoformat.json

In [5]:
# convert the dataset to coco format
!python ./tools/convert_datasets/visdronemot/visdronemot2coco.py -d ./data/VisDrone_MOT/test

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
100%|█████████████████████████████████████████████| 7/7 [00:01<00:00,  6.38it/s]


## 修改训练配置

In [1]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/det/faster-rcnn_r50_fpn_4e_visdrone_mot.py')
# cfg.data_root = 'data/VisDrone_MOT/'
# cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

# cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.work_dir = './tutorial_exps/detector'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = "cuda"
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [0]
print(f'Config:\n{cfg.pretty_text}')

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

## 检测器训练&验证

In [2]:
import os.path as osp

from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmdet.models import build_detector as build_model

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
model = build_model(cfg.model.detector)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES
train_model(model, datasets, cfg, validate=True)

2022-11-08 15:58:34,446 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'}
2022-11-08 15:58:34,447 - mmcv - INFO - load model from: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2022-11-08 15:58:34,448 - mmcv - INFO - load checkpoint from http path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2022-11-08 15:58:34,539 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([3, 1024]).
size mism

loading annotations into memory...
Done (t=1.26s)
creating index...
index created!


/mmdetection/mmdet/utils/compat_config.py:30: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
2022-11-08 15:58:38,112 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-11-08 15:58:38,288 - mmdet - INFO - Start running, host: root@5f2d80d40d9d, work_dir: /workdir/tutorial_exps/detector
2022-11-08 15:58:38,289 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
bef

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
2022-11-08 15:58:51,777 - mmdet - INFO - Epoch [1][50/10984]	lr: 9.902e-03, eta: 3:16:26, time: 0.269, data_time: 0.047, memory: 2946, loss_rpn_cls: 0.1347, loss_rpn_bbox: 0.0486, loss_cls: 0.3227, acc: 89.5723, loss_bbox: 0.1123, loss: 0.6182
2022-11-08 15:59:02,732 - mmdet - INFO - Epoch [1][100/10984]	lr: 1.980e-02, eta: 2:58:07, time: 0.219, data_time: 0.004, memory: 2946, loss_rpn_cls: 0.0925, loss_rpn_bbox: 0.0273, loss_cls: 0.2240, acc: 93.9180, loss_bbox: 0.0913, loss: 0.4350
2022-11-08 15:59:13,901 - mmdet - INFO - Epoch [1][150/10984]	lr: 2.

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 2846/2846, 19.3 task/s, elapsed: 147s, ETA:     0s

2022-11-08 18:57:39,567 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.08s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=39.62s).
Accumulating evaluation results...


2022-11-08 18:58:21,531 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.337
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.050
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.053
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.216
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.216
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.216
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.105
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.262
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.86s).


# ReID模块

## 数据集处理（转化为ReID数据集）

由训练集分出一部分作为测试集

In [12]:
!rm -r ./data/VisDrone_MOT/train/reid 
!rm -r ./data/VisDrone_MOT/train/reid_meta

In [13]:
!python ./tools/convert_datasets/visdronemot/visdronemot2reid.py \
    -d ./data/VisDrone_MOT/train \
    --get-images 

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
corp reid images...
100%|███████████████████████████████████████████| 56/56 [10:16<00:00, 11.01s/it]


In [6]:
!python ./tools/convert_datasets/visdronemot/visdronemot2reid.py \
    -d ./data/VisDrone_MOT/train \
    --get-labels \
    --min-thr=500 --max-thr=20

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
100%|████████████████████████████████████| 2620/2620 [00:00<00:00, 17438.25it/s]
num classes: 76, lines of train_labels: 1368, lines of test_labels: 152


## 修改训练配置

In [4]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/reid/resnet50_b32x8_VisDroneMOT.py')
# cfg.data_root = 'data/VisDrone_MOT_tiny/'
# cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

# cfg.data.train.data_prefix = cfg.data.train.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.val.data_prefix = cfg.data.val.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.test.data_prefix = cfg.data.test.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

# learning policy
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=200,
    warmup_ratio=1.0 / 200,
    step=[1])
cfg.total_epochs = 10

cfg.work_dir = './tutorial_exps/reid'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = "cuda"
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [0]
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'ReIDDataset'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadMultiImagesFromFile', to_float32=True),
    dict(
        type='SeqResize',
        img_scale=(128, 256),
        share_params=False,
        keep_ratio=False,
        bbox_clip_border=False,
        override=False),
    dict(
        type='SeqRandomFlip',
        share_params=False,
        flip_ratio=0.5,
        direction='horizontal'),
    dict(
        type='SeqNormalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='VideoCollect', keys=['img', 'gt_label']),
    dict(type='ReIDFormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', img_scale=(128, 256), keep_ratio=False),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),

## ReID模块训练&验证

In [5]:
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmtrack.models import build_reid as build_model


model = build_model(cfg.model.reid)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

train_model(model, datasets, cfg, validate=True)

2022-11-09 13:05:04,452 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth'}
2022-11-09 13:05:04,453 - mmcv - INFO - load model from: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2022-11-09 13:05:04,453 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2022-11-09 13:05:04,518 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fc.weight, head.fc.bias

missing keys in source state_dict: head.fcs.0.fc.weight, head.fcs.0.fc.bias, head.fcs.0.bn.weight, head.fcs.0.bn.bias, head.fcs.0.bn.running_mean, head.fcs.0.bn.running_var, head.fc_out.weight, head.fc_out.bias, head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.

2022-11-09 13:07:58,915 - mmdet - INFO - Epoch [1][1300/1368]	lr: 1.000e-01, eta: 0:27:39, time: 0.132, data_time: 0.004, memory: 2062, triplet_loss: 0.5176, ce_loss: 3.3881, top-1: 9.8750, loss: 3.9057
2022-11-09 13:08:05,534 - mmdet - INFO - Epoch [1][1350/1368]	lr: 1.000e-01, eta: 0:27:32, time: 0.132, data_time: 0.003, memory: 2062, triplet_loss: 0.4367, ce_loss: 3.2455, top-1: 11.1875, loss: 3.6822
2022-11-09 13:08:07,905 - mmdet - INFO - Saving checkpoint at 1 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 152/152, 125.4 task/s, elapsed: 1s, ETA:     0s

2022-11-09 13:08:09,610 - mmdet - INFO - Epoch(val) [1][152]	mAP: 0.4070
2022-11-09 13:08:18,429 - mmdet - INFO - Epoch [2][50/1368]	lr: 1.000e-02, eta: 0:27:20, time: 0.175, data_time: 0.046, memory: 2062, triplet_loss: 0.4288, ce_loss: 2.9973, top-1: 14.9375, loss: 3.4261
2022-11-09 13:08:25,051 - mmdet - INFO - Epoch [2][100/1368]	lr: 1.000e-02, eta: 0:27:12, time: 0.132, data_time: 0.003, memory: 2062, triplet_loss: 0.3553, ce_loss: 2.7323, top-1: 19.1875, loss: 3.0876
2022-11-09 13:08:31,672 - mmdet - INFO - Epoch [2][150/1368]	lr: 1.000e-02, eta: 0:27:05, time: 0.132, data_time: 0.003, memory: 2062, triplet_loss: 0.2929, ce_loss: 2.5540, top-1: 24.6875, loss: 2.8469
2022-11-09 13:08:38,297 - mmdet - INFO - Epoch [2][200/1368]	lr: 1.000e-02, eta: 0:26:58, time: 0.133, data_time: 0.003, memory: 2062, triplet_loss: 0.3026, ce_loss: 2.6224, top-1: 23.6875, loss: 2.9250
2022-11-09 13:08:44,924 - mmdet - INFO - Epoch [2][250/1368]	lr: 1.000e-02, eta: 0:26:51, time: 0.133, data_time: 0.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 152/152, 122.4 task/s, elapsed: 1s, ETA:     0s

...

2022-11-09 13:29:45,702 - mmdet - INFO - Epoch(val) [8][152]	mAP: 0.4920
2022-11-09 13:29:54,522 - mmdet - INFO - Epoch [9][50/1368]	lr: 1.000e-02, eta: 0:05:55, time: 0.175, data_time: 0.045, memory: 2062, triplet_loss: 0.3237, ce_loss: 1.7820, top-1: 48.0625, loss: 2.1057
2022-11-09 13:30:01,154 - mmdet - INFO - Epoch [9][100/1368]	lr: 1.000e-02, eta: 0:05:49, time: 0.133, data_time: 0.003, memory: 2062, triplet_loss: 0.3215, ce_loss: 1.7899, top-1: 49.6875, loss: 2.1115
2022-11-09 13:30:07,786 - mmdet - INFO - Epoch [9][150/1368]	lr: 1.000e-02, eta: 0:05:42, time: 0.133, data_time: 0.003, memory: 2062, triplet_loss: 0.2811, ce_loss: 1.6010, top-1: 53.3125, loss: 1.8822
2022-11-09 13:30:14,415 - mmdet - INFO - Epoch [9][200/1368]	lr: 1.000e-02, eta: 0:05:36, time: 0.133, data_time: 0.003, memory: 2062, triplet_loss: 0.2684, ce_loss: 1.6149, top-1: 53.5625, loss: 1.8833
2022-11-09 13:30:21,036 - mmdet - INFO - Epoch [9][250/1368]	lr: 1.000e-02, eta: 0:05:29, time: 0.132, data_time: 0.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 152/152, 126.1 task/s, elapsed: 1s, ETA:     0s

2022-11-09 13:32:50,893 - mmdet - INFO - Epoch(val) [9][152]	mAP: 0.4920
2022-11-09 13:32:59,691 - mmdet - INFO - Epoch [10][50/1368]	lr: 1.000e-02, eta: 0:02:54, time: 0.175, data_time: 0.045, memory: 2062, triplet_loss: 0.2954, ce_loss: 1.3975, top-1: 60.7500, loss: 1.6930
2022-11-09 13:33:06,316 - mmdet - INFO - Epoch [10][100/1368]	lr: 1.000e-02, eta: 0:02:48, time: 0.133, data_time: 0.003, memory: 2062, triplet_loss: 0.3476, ce_loss: 1.6066, top-1: 55.5000, loss: 1.9542
2022-11-09 13:33:12,940 - mmdet - INFO - Epoch [10][150/1368]	lr: 1.000e-02, eta: 0:02:41, time: 0.132, data_time: 0.003, memory: 2062, triplet_loss: 0.2770, ce_loss: 1.4224, top-1: 60.4375, loss: 1.6994
2022-11-09 13:33:19,562 - mmdet - INFO - Epoch [10][200/1368]	lr: 1.000e-02, eta: 0:02:34, time: 0.132, data_time: 0.003, memory: 2062, triplet_loss: 0.2601, ce_loss: 1.3256, top-1: 62.6250, loss: 1.5857
2022-11-09 13:33:26,184 - mmdet - INFO - Epoch [10][250/1368]	lr: 1.000e-02, eta: 0:02:28, time: 0.132, data_tim

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 152/152, 125.3 task/s, elapsed: 1s, ETA:     0s

2022-11-09 13:35:56,089 - mmdet - INFO - Epoch(val) [10][152]	mAP: 0.5910


# 验证跟踪模型（检测器+跟踪）

In [1]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py')
# cfg.data_root = 'data/VisDrone_MOT_tiny/'
# cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

# cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
# cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_10.pth'

cfg.work_dir = './tutorial_exps'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = 'cuda'
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [0]
cfg.data.test.test_mode = True
print(f'Config:\n{cfg.pretty_text}')

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

In [2]:
from mmtrack.datasets import build_dataloader
from mmtrack.apis import init_model
from mmcv.parallel import MMDataParallel
from mmtrack.apis import single_gpu_test
from mmtrack.datasets import build_dataset

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

# build the model and load checkpoint
model = init_model(cfg)

model = MMDataParallel(model, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

eval_kwargs = cfg.get('evaluation', {}).copy()
# hard-code way to remove EvalHook args
eval_hook_args = [
    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
    'rule', 'by_epoch'
]
for key in eval_hook_args:
    eval_kwargs.pop(key, None)
eval_kwargs.update(dict(metric=['track']))
metric = dataset.evaluate(outputs, **eval_kwargs)
print(metric)

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


2022-11-09 14:16:48,512 - mmtrack - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-11-09 14:16:48,513 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-11-09 14:16:48,514 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-11-09 14:16:50,194 - mmtrack - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_10.pth'}
2022-11-09 14:16:50,195 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_10.pth
2022-11-09 14:16:50,196 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_10.pth


[                                ] 2/2846, 4.2 task/s, elapsed: 0s, ETA:   683s

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 2846/2846, 10.7 task/s, elapsed: 265s, ETA:     0s---CLEAR MOT Evaluation---
Accumulating...
Evaluating...
Rendering...

            IDF1   MOTA  MOTP    FP    FN IDSw  Rcll  Prcn MT PT  ML   FM
pedestrian 45.0%  13.3% 0.738  7547 10004  411 51.7% 58.7% 26 36  60  549
people     16.8% -29.6% 0.676  4922  8649  310 19.2% 29.5%  7 25  40  274
OVERALL    36.2%  -1.3% 0.728 12469 18653  721 40.6% 50.6% 33 61 100  823
AVERAGE    30.9%  -8.2% 0.707 12469 18653  721 35.5% 44.1% 33 61 100  823
Evaluation finishes with 2.56 s.
{'IDF1': 0.362, 'MOTA': -0.013, 'MOTP': 0.728, 'FP': 12469, 'FN': 18653, 'IDSw': 721, 'Rcll': 0.406, 'Prcn': 0.506, 'MT': 33, 'PT': 61, 'ML': 100, 'FM': 823, 'track_OVERALL_copypaste': '0.362 -0.013 0.728 12469 18653 721 0.406 0.506 33 61 100 823 ', 'track_AVERAGE_copypaste': '0.309 -0.082 0.707 12469 18653 721 0.355 0.441 33 61 100 823 '}


# 推理（检测+跟踪）

In [5]:
# run mot demo
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model
# mot_config = './configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py'
cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py')
cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_2.pth'
input_video = './demo/visdrone_demo.mp4'
imgs = mmcv.VideoReader(input_video)
# build the model from a config file
# mot_model = init_model(mot_config, device='cuda:0')
mot_model = init_model(cfg)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name
# test and show/save the images
for i, img in enumerate(imgs):
    result = inference_mot(mot_model, img, frame_id=i)
    mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()

output = './demo/visdrone_demo_result.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-11-08 20:01:15,845 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-11-08 20:01:15,846 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 20:01:15,846 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 20:01:16,056 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-11-08 20:01:16,056 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 20:01:16,057 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 20:01:16,186 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,186 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps

2022-11-08 20:01:16,206 - mmcv - INFO - 
detector.backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,210 - mmcv - INFO - 
detector.backbone.layer2.1.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,210 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,210 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,211 - mmcv - INFO - 
detector.backbone.layer2.1.conv2.weight - torch.Size([128, 128, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,212 - mmcv - INFO - 
detector.backbone.layer2.1.bn2.weight - torch.Size([128]): 
PretrainedInit: load from ./tutor

2022-11-08 20:01:16,230 - mmcv - INFO - 
detector.backbone.layer3.1.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,230 - mmcv - INFO - 
detector.backbone.layer3.2.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,231 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,231 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,231 - mmcv - INFO - 
detector.backbone.layer3.2.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,232 - mmcv - INFO - 
detector.backbone.layer3.2.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exp

2022-11-08 20:01:16,260 - mmcv - INFO - 
detector.backbone.layer4.0.downsample.1.bias - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,261 - mmcv - INFO - 
detector.backbone.layer4.1.conv1.weight - torch.Size([512, 2048, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,261 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,261 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,262 - mmcv - INFO - 
detector.backbone.layer4.1.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,262 - mmcv - INFO - 
detector.backbone.layer4.1.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tut

2022-11-08 20:01:16,286 - mmcv - INFO - 
detector.roi_head.bbox_head.shared_fcs.1.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 20:01:16,286 - mmcv - INFO - 
reid.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,287 - mmcv - INFO - 
reid.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,287 - mmcv - INFO - 
reid.backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,287 - mmcv - INFO - 
reid.backbone.layer1.0.conv1.weight - torch.Size([64, 64, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,288 - mmcv - INFO - 
reid.backbone.layer1.0.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,289 - mmcv - INFO - 

2022-11-08 20:01:16,313 - mmcv - INFO - 
reid.backbone.layer2.1.bn2.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,313 - mmcv - INFO - 
reid.backbone.layer2.1.conv3.weight - torch.Size([512, 128, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,313 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,314 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,314 - mmcv - INFO - 
reid.backbone.layer2.2.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,315 - mmcv - INFO - 
reid.backbone.layer2.2.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,315 

2022-11-08 20:01:16,340 - mmcv - INFO - 
reid.backbone.layer3.2.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,340 - mmcv - INFO - 
reid.backbone.layer3.3.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,341 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,341 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,341 - mmcv - INFO - 
reid.backbone.layer3.3.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,342 - mmcv - INFO - 
reid.backbone.layer3.3.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,34

2022-11-08 20:01:16,361 - mmcv - INFO - 
reid.backbone.layer4.2.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,361 - mmcv - INFO - 
reid.backbone.layer4.2.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,362 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,362 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,363 - mmcv - INFO - 
reid.backbone.layer4.2.conv3.weight - torch.Size([2048, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,363 - mmcv - INFO - 
reid.backbone.layer4.2.bn3.weight - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 20:01:16,36

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 109/109, 9.4 task/s, elapsed: 12s, ETA:     0s
 making the output video at ./demo/visdrone_demo_result.mp4 with a FPS of 10.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 109/109, 63.7 task/s, elapsed: 2s, ETA:     0s
